**Previous book**: <a href='./03_data_subsetting.ipynb'>[Data Subsetting]</a>

## Part 4: ETL with API data and enrichment

**Note**: This notebook expects <code>stage3_serials.csv</code> from the <code>03_data_subsetting.ipynb</code> notebook!

At this stage, the focus of the project has narrowed to television serials. However, the original dataset was scraped in 2021, and as such, a lot of the information is now out-dated - particularly the <code>end_year</code> column. Many of the entries labelled as <i>ongoing</i> have since concluded.

To address this issue, we need to consult an external source. We will make use of an external API to create a simple ETL pipeline that will load data from a database into a local dataset. This new dataset can then be used to update and enrich our existing dataset with the enriched dataset serving as the foundation for future analysis.

**Input:** <code>/data/interim/stage3_serials.csv</code> and <code>/data/final/api_data.csv</code>
<br>
**Output:** <code>/data/interim/stage4_enriched_serial_data.csv</code>
### Initial imports
Start by importing the <code>stage3_serials.csv</code> file created in Part 3:

In [1]:
import numpy as np
import pandas as pd
import requests

# start_year acts more like a label than a true integer
df = pd.read_csv("../data/interim/stage3_serials.csv",
                 dtype={"start_year": "str", "votes": "Int64","run_time": "Int64"})

### Extract, Transform, Load (ETL)
<b>ETL</b> (Extract, Transform, Load) is the process of extracting data, transforming it into the desired structure, and then loading it into storage.

For this project, we will use the API provided by <a href="https://www.tvmaze.com/">TVMaze</a>, a free user-driven television database. The <a href="https://www.tvmaze.com/api">TVMaze API</a> (or Application Programming Interface) returns information about a given TV show in JSON format. For simplicity, we will use the <b>show single search</b>. This end-point returns either a single show if the search text matches an entry, otherwise it returns nothing.

There are some limitation to this approach. Care must be taken to ensure that the information returned is for the correct serial. As shown earlier, it is possible for television serials to share the some name. It also possible that some entries may not be found in the TVMaze database, in this case, we won't be able to update the information for these entries.

#### Extracting data from an external source using an API
For the first step of the ETL process, extract, we write a function that gets the information from the end-point using the requests package. This function takes a list of shows and returns a list of JSON containing the information about every show. If no such show is found, a blank entry is added as a placeholder so that the data frames are the same length. This will be important later.

In [2]:
def extract_api_data(shows):
    json_list = []
    for show in shows:
        url = f"https://api.tvmaze.com/singlesearch/shows?q={show.replace(' ', '%20')}"
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            json_list.append(data)
        else:
            # in the json, 'average' is nested under 'rating', so we copy the structure here
            json_list.append({"name": show, "rating": {"average": None}})
    print("Extract ran successfully")
    return json_list

The titles of the entire dataset can be returned as a list:

In [3]:
television_shows = list(df["title"])

Rather than use the entire dataset, we can demonstrate how the function works with a toy dataset containing the first five entries:

In [4]:
demonstration = extract_api_data(television_shows[0:5])
demonstration[0]

Extract ran successfully


{'id': 43428,
 'url': 'https://www.tvmaze.com/shows/43428/masters-of-the-universe-revelation',
 'name': 'Masters of the Universe: Revelation',
 'type': 'Animation',
 'language': 'English',
 'genres': ['Action', 'Adventure', 'Fantasy'],
 'status': 'Ended',
 'runtime': None,
 'averageRuntime': 26,
 'premiered': '2021-07-23',
 'ended': '2021-11-23',
 'officialSite': 'https://www.netflix.com/title/81154670',
 'schedule': {'time': '', 'days': []},
 'rating': {'average': 6.6},
 'weight': 94,
 'network': None,
 'webChannel': {'id': 1,
  'name': 'Netflix',
  'country': None,
  'officialSite': 'https://www.netflix.com/'},
 'dvdCountry': None,
 'externals': {'tvrage': None, 'thetvdb': 368321, 'imdb': 'tt10826054'},
 'image': {'medium': 'https://static.tvmaze.com/uploads/images/medium_portrait/365/914541.jpg',
  'original': 'https://static.tvmaze.com/uploads/images/original_untouched/365/914541.jpg'},
 'summary': "<p>A radical return to Eternia, <b>Masters of the Universe: Revelation</b> is a dir

Above, we can see the structure of the JSON returned from the API.

#### Transforming data collected using an API

The next step of the ETL process is transformation. In this case, the transformation function will contain two smaller functions.

These functions will:

<ol>
    <li>Save the required information from the JSON to a new dataframe.</li>
    <li>Change the information to better match the format of the original IMDb dataframe.</li>
</ol>

From the JSON above, we can identify the columns of interest for us:
<ul>
    <li><b>name</b> - title of the show.</li>
    <li><b>status</b> - is the show currently running?</li>
    <li><b>AverageRuntime</b> - the average length of an episode.</li>
    <li><b>premiered</b> - the date when the show first aired.</li>
    <li><b>ended</b> - the date when the final episode aired.</li>
    <li><b>average</b> nested under <b>rating</b> - the average rating of the show.</li>
    <li><b>summary</b> - a brief description of the serial.</li>
    <li><b>language</b> - the main language spoken in the serial.</li>
</ul>

The following function retrieves this information from the JSON list, and writes it to a new data frame with column names matching the original data:

In [5]:
def write_df(json_data):
    normalised_data = []
    for row in json_data:
        title = row.get("name")
        language = row.get("language")
        status = row.get("status")
        run_time = row.get("averageRuntime")
        start_year = row.get("premiered")
        end_year = row.get("ended")
        rating = row.get("rating").get("average")
        summary = row.get("summary")
        normalised_data.append([title, start_year, end_year, language, run_time, rating, summary, status])
    new_df = pd.DataFrame(normalised_data)
    new_df.columns = ["title", "start_year", "end_year", "language", "run_time", "rating", "summary", "status"]
    return new_df

With a new data frame created, we now need to adapt the information to better match our original data.

To do this, we make the following changes:
<ul>
    <li><code>start_year</code> and <code>end_year</code> should just be a year. Use a regex pattern to extract the year (alternatively: <code>dt.strftime.("%Y")</code>)</li>
    <li>Remove HTML tags from <code>summary</code> using regex.</li>
    <li>Cast the <code>status</code> column as a category.</li>
</ul>

The following function makes the required changes:

In [6]:
def clean_api_data(a_df):
    date_regex = r"(\d{4})-\d{2}-\d{2}"
    a_df["start_year"] = a_df["start_year"].str.replace(date_regex, r"\1", regex=True)
    a_df["end_year"] = a_df["end_year"].str.replace(date_regex, r"\1", regex=True)
    a_df["summary"] = a_df["summary"].str.replace(r"</?\w+>", "", regex=True)
    a_df["status"] = a_df["status"].astype("category")
    return a_df

Combine the two functions into a larger <code>transform_api_data</code> function:

In [7]:
def transform_api_data(json_data):
    api_df = write_df(json_data)
    print("Data written to data frame")
    clean_api_data(api_df)
    print("Data cleaned")
    return api_df

Run the transform function with the demonstration data to see if it work correctly:

In [8]:
demonstration_df = transform_api_data(demonstration)
demonstration_df

Data written to data frame
Data cleaned


,title,start_year,end_year,language,run_time,rating,summary,status
0,Masters of the Universe: Revelation,2021,2021,English,26,6.6,"A radical return to Eternia, Masters of the Un...",Ended
1,The Walking Dead,2010,2022,English,62,7.9,The Walking Dead tells the story of the months...,Ended
2,Rick and Morty,2013,None,English,30,8.8,"Rick is a mentally gifted, but sociopathic and...",Running
3,Outer Banks,2020,None,English,52,7.0,"On an island of haves and have-nots, teen John...",Running
4,Dexter,2006,2013,English,60,8.4,"He's smart, he's good looking, and he's got a ...",Ended


As shown above, both the extract and the transformation functions work as intended.

#### Load API data
The last step of the ETL process is loading. In this case, we will simply save the data frame as a CSV file to a given file path.

In [9]:
def load_api_data(a_df, filepath):
    a_df.to_csv(filepath, index=False)
    print(f"Data written to file: {filepath}")

It is not necessary to show how this function works with the demonstration dataset.

All of the ETL functions together form a data pipeline used to extract data from the TVMaze, transform the data to the desired structure, and save it locally. This dataset can then be used for analysis or to enrich our existing dataset.

### Enriching the original data
The dataset, <code>api_data.csv</code>, is the result of running the ETL pipeline externally with all the titles in the IMDb dataset (retrieved 27th May 2025).

Load the data and examine the structure:

In [10]:
api_data = pd.read_csv("../data/final/api_data.csv", dtype={"start_year": "str", "end_year": "str", "run_time": "Int64"})

In [11]:
api_data.head()

,title,start_year,end_year,language,run_time,rating,summary,status
0,Masters of the Universe: Revelation,2021,2021,English,26,6.6,"A radical return to Eternia, Masters of the Un...",Ended
1,The Walking Dead,2010,2022,English,62,7.9,The Walking Dead tells the story of the months...,Ended
2,Rick and Morty,2013,NaN,English,30,8.8,"Rick is a mentally gifted, but sociopathic and...",Running
3,Outer Banks,2020,NaN,English,52,7.0,"On an island of haves and have-nots, teen John...",Running
4,Dexter,2006,2013,English,60,8.4,"He's smart, he's good looking, and he's got a ...",Ended


Examine the missing values in the dataset:

In [12]:
api_data.isna().sum()

title           0
start_year    192
end_year      531
language      204
run_time      200
rating        825
summary       198
status        185
dtype: int64

It appears that some entries are missing. This may correspond to titles in the original dataset that were not present in the TVMaze database, and thus are missing from the dataset.

#### A note about the approach
My original approach to enriching the dataset was to merge the original dataset with the new dataset from the API on the <code>title</code> and <code>start_year</code> columns (the combination shown previously to be able to uniquely identify all records). However, there were some unforeseen problems with this.

For example, looking at a record from the IMDb dataset:

In [13]:
df.iloc[31]

title                                        Shingeki no kyojin
start_year                                                 2013
end_year                                                   2022
genre                              Animation, Action, Adventure
summary       After his hometown is destroyed and his mother...
rating                                                      9.0
votes                                                    242582
run_time                                                     24
actors        Yûki Kaji, Marina Inoue, Josh Grelle, Yui Ishi...
Name: 31, dtype: object

And the same record from the TVMaze dataset:

In [14]:
api_data.iloc[31]

title                                           Attack on Titan
start_year                                                 2013
end_year                                                   2023
language                                               Japanese
run_time                                                     25
rating                                                      8.9
summary       Known in Japan as Shingeki no Kyojin, many yea...
status                                                    Ended
Name: 31, dtype: object

These records both refer to the same show. The problem is that the IMDb title is the romanised form of the original Japanese title, whereas TVMaze's API returns the English name. This mismatch would prevent the two records from merging under the original approach. This problem would also extend to differences in capitalisation or other minor differences in titles.

This necessitated a new approch where more of the data could be preserved.

#### New approach

Rather than match records based on title, we can match records based their on position in the dataset. This is why titles not found with the API were still added to the dataset.

First make sure that we have the same number of rows in each dataset:

In [15]:
assert len(api_data) == len(df)

Both dataframes contain the same number of rows.

To avoid matching serials with similar titles, we can ensure that <code>start_year</code> is consistent between the two:

In [16]:
year_mask = df["start_year"] == api_data["start_year"]

Filter API data to remove any rows that do not match the start year:

In [17]:
api_data_filtered = api_data.where(year_mask)
# left join to retain all the original records
enriched_data = df.join(api_data_filtered, how="left", lsuffix="_imdb", rsuffix="_tvmaze")

Merge the filtered <code>api_data</code> with the <code>df</code> to get the amalgamated dataset:

In [18]:
enriched_data.head(3)

,title_imdb,start_year_imdb,end_year_imdb,genre,summary_imdb,rating_imdb,votes,run_time_imdb,actors,title_tvmaze,start_year_tvmaze,end_year_tvmaze,language,run_time_tvmaze,rating_tvmaze,summary_tvmaze,status
0,Masters of the Universe: Revelation,2021,Ongoing,"Animation, Action, Adventure",The war for Eternia begins again in what may b...,5.0,17870,25,"Chris Wood, Sarah Michelle Gellar, Lena Headey...",Masters of the Universe: Revelation,2021,2021,English,26,6.6,"A radical return to Eternia, Masters of the Un...",Ended
1,The Walking Dead,2010,2022,"Drama, Horror, Thriller",Sheriff Deputy Rick Grimes wakes up from a com...,8.2,885805,44,"Andrew Lincoln, Norman Reedus, Melissa McBride...",The Walking Dead,2010,2022,English,62,7.9,The Walking Dead tells the story of the months...,Ended
2,Rick and Morty,2013,Ongoing,"Animation, Adventure, Comedy",An animated series that follows the exploits o...,9.2,414849,23,"Justin Roiland, Chris Parnell, Spencer Grammer...",Rick and Morty,2013,NaN,English,30,8.8,"Rick is a mentally gifted, but sociopathic and...",Running


#### Enrichment
The main purpose of enriching the dataset is to update the <code>end_year</code> column, so we will start with that.

Set all the entries with <code>Ongoing</code> to be NaN:

In [19]:
enriched_data.loc[enriched_data["end_year_imdb"] == "Ongoing", "end_year_imdb"] = np.nan

Update the missing values in <code>end_year</code> with the ones retrieved from TVMaze. 

**NOTE**: If the value for <code>end_year</code> is not missing, this value will not be updated even if it is incorrect. This may cause disagreements between <code>end_year</code> and <code>status</code> in some cases, where a TV show that has ended according to the IMDb data is still shown as running based on the TVMaze data. See: *Family Guy* as an example of this behaviour.

In [20]:
enriched_data["end_year_imdb"] = enriched_data["end_year_imdb"].fillna(value=enriched_data["end_year_tvmaze"])

##### Status
To keep formatting and data types consistent across fields, we can impute <code>status</code> based on <code>end_year</code>.

**Note**: This approach simplifies the category somewhat. The TVMaze data has an additional category 'To Be Determined' where there is no <code>end_year</code>, but the show is not necessarily continuing. One possible solution is to drop this column and calculate the status dynamically. However imputing values based on <code>end_year</code> is the most practical approach and conserves the most data.

First, check the how much data overwriting 'To Be Determined' would affect:

In [21]:
enriched_data["status"].value_counts()

status
Ended               1094
To Be Determined     149
Running              134
Name: count, dtype: int64

Out of the non-missing data, 149 out of 1377 (or 10.8%) is 'To Be Determined'. This is quite a sizeable proportion of the data, so we should continue with the imputation strategy:

In [22]:
enriched_data.loc[(enriched_data["end_year_imdb"].isna()) & (enriched_data["status"].isna()), "status"] = "Running"
enriched_data.loc[(~enriched_data["end_year_imdb"].isna()) & (enriched_data["status"].isna()), "status"] = "Ended"


##### Runtime
Take the opportunity to update values for <code>run_time</code>. This will reinstate the extreme values removed earlier, if they were indeed correct.

In [23]:
enriched_data["run_time_imdb"] = enriched_data["run_time_imdb"].fillna(value=enriched_data["run_time_tvmaze"])

#### Clean up
Finally, rename the original columns back to their original names:

In [24]:
new_names = {
    "title_imdb": "title",
    "start_year_imdb": "start_year",
    "end_year_imdb": "end_year",
    "run_time_imdb": "run_time"
}

enriched_data = enriched_data.rename(new_names, axis=1)

Select and order the remaining columns.

In [25]:
ordered_columns = [
    "title",
    "start_year",
    "end_year",
    "genre",
    "summary_tvmaze",
    "summary_imdb",
    "rating_tvmaze",
    "rating_imdb",
    "votes",
    "run_time",
    "language",
    "actors",
    "status"
]

enriched_data = enriched_data.loc[:, ordered_columns]

### Enrichment summary

Check the changes made to the data frame:



In [26]:
enriched_data.head()

,title,start_year,end_year,genre,summary_tvmaze,summary_imdb,rating_tvmaze,rating_imdb,votes,run_time,language,actors,status
0,Masters of the Universe: Revelation,2021,2021,"Animation, Action, Adventure","A radical return to Eternia, Masters of the Un...",The war for Eternia begins again in what may b...,6.6,5.0,17870,25,English,"Chris Wood, Sarah Michelle Gellar, Lena Headey...",Ended
1,The Walking Dead,2010,2022,"Drama, Horror, Thriller",The Walking Dead tells the story of the months...,Sheriff Deputy Rick Grimes wakes up from a com...,7.9,8.2,885805,44,English,"Andrew Lincoln, Norman Reedus, Melissa McBride...",Ended
2,Rick and Morty,2013,NaN,"Animation, Adventure, Comedy","Rick is a mentally gifted, but sociopathic and...",An animated series that follows the exploits o...,8.8,9.2,414849,23,English,"Justin Roiland, Chris Parnell, Spencer Grammer...",Running
3,Outer Banks,2020,NaN,"Action, Crime, Drama","On an island of haves and have-nots, teen John...",A group of teenagers from the wrong side of th...,7.0,7.6,25858,50,English,"Chase Stokes, Madelyn Cline, Madison Bailey, J...",Running
4,Dexter,2006,2013,"Crime, Drama, Mystery","He's smart, he's good looking, and he's got a ...","By day, mild-mannered Dexter is a blood-spatte...",8.4,8.6,665387,53,English,"Michael C. Hall, Jennifer Carpenter, David Zay...",Ended


In this part, we constructed an API to obtain data from an external source that could be used to enhance the underlying data. We used the new data frame to fill in missing values and add additional columns that may be useful in further analysis.

The ETL pipeline can be adapted to retrieve information about different shows, or to run on a schedule by using a data orchestration tool such as <i>Apache Airflow</i>.

**Note I**: the goal of this part was to enhance the original data, not replace it. Therefore only missing data was added to the original data frame and new columns added to the original columns.

**Note II**: I have noticed that some missing entries may be found with some modifications:

There are two many cases for these:

**Firstly, the title in the IMDb dataset is too different from the title in the TVMaze dataset.**

For example, in the original dataset we have <i>NCIS: Naval Criminal Investigation Service</i>. This title does not appear in the TVMaze database, however <i>NCIS</i> does. This title and others may be found if we clipped titles that contained <code>:</code> to just the first part of their title. However, this may have unintended consequences, so both versions should be tried to be sure.

**Secondly, the start years differ between the datasets**

<i>Adventure Time</i> exists in both datasets, but haven't been matched because <code>start_year</code> differs (2010 vs 2008). This can be addressed by allowing more 'fuzzy' matching on <code>start_year</code>. As before, this may lead to unintended consequences, so fuzzy matches should be checked before continuing.

Overall, a decent amount of data has been found, and can be adequately used for future analysis.


Save the newly enhanced and updated data frame as <code>stage4_enriched_serial_data.csv</code>. This file will be used in the next part, <i>data normalisation</i>.

In [27]:
enriched_data.to_csv("../data/interim/stage4_enriched_serial_data.csv", index=False)

### Navigation
**Previous book**: <a href='./03_data_subsetting.ipynb'>[Data Subsetting]</a>

**Next book**: <a href='./05_normalisation.ipynb'>[Normalisation]</a>